#### Script to check rf and compute the visuomemory index block 1
- First, check if neuron responding to the task by comparing activity during trial vs baseline
- Second, check whether the rf is in in or out
- Third, compute the visuomemory index

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
import pandas as pd
from typing import Dict
from collections import defaultdict
from ephysvibe.trials.spikes import firing_rate,sp_constants,plot_raster
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.structures.bhv_data import BhvData
from ephysvibe.structures.spike_data import SpikeData
from ephysvibe.analysis import raw_ch,circular_stats
from ephysvibe.task import task_constants,def_task
import os 
seed=2023

### Def functions

In [17]:
def get_neurons_info(sp_samples:np.ndarray,fix_t:int,neuron_type:np.ndarray,task:pd.DataFrame,dur_v:int,st_m:int,end_m:int,st_t:int,end_t:int,min_trials:int=5,n_spikes_sec:int=5)->pd.DataFrame:
    trial_dur = sp_samples.shape[2]
    neurons_info:Dict[str,list] = defaultdict(list)
    i_good,i_mua, n_type = 0,0,0
    for i_neuron,type_neuron in enumerate(neuron_type):
        # check and count type of unit
        if type_neuron == 'good':
            i_good+=1
            n_type = i_good
        elif type_neuron == 'mua':
            i_mua+=1
            n_type = i_mua
        max_in_out = np.array([0,0])
        larger,v_larger,m_larger,idx_p_min = np.array([False,False]),np.array([False,False]),np.array([False,False]),np.array([0,0])
        t_larger=np.nan
        p_in_out,p_v,p_m,vm_index = np.array([None,None]),np.array([None,None]),np.array([None,None]),np.nan
        tr_min,tr_max,mean_mem_all,mean_visual_all = np.array([np.nan,np.nan]),np.array([np.nan,np.nan]),np.array([np.nan,np.nan]),np.array([np.nan,np.nan])
        for i,in_out in enumerate(["in","out"]): # iterate by code'
            trial_idx = task[(task['i_neuron']==i_neuron)&(task['in_out']==in_out)&(task['sample']!='o0_c0')]["trial_idx"]
            trial_idx = trial_idx[(sp_samples[trial_idx,i_neuron].sum(axis=1)>0)]
            n_tr = len(trial_idx)
            true_in_out = 'in'
            if n_tr > 3:
                mean_visual = sp_samples[trial_idx,i_neuron,fix_t+50:fix_t+dur_v].mean(axis=1)
                mean_mem = sp_samples[trial_idx,i_neuron,fix_t+st_m:fix_t+end_m].mean(axis=1)
                mean_test = sp_samples[trial_idx,i_neuron,fix_t+st_t:fix_t+end_t].mean(axis=1)
                mean_bl = sp_samples[trial_idx,i_neuron,:fix_t].mean(axis=1)
                v_larger[i] = mean_bl.mean()<mean_visual.mean()
                m_larger[i] = mean_bl.mean()<mean_mem.mean()
                t_larger = mean_bl.mean()<mean_test.mean()
                larger[i]= v_larger[i] or m_larger[i] or t_larger

                p_v[i] = stats.ttest_rel(mean_bl, mean_visual)[1]
                p_m[i] = stats.ttest_rel(mean_bl, mean_mem)[1]
                p_t = stats.ttest_rel(mean_bl, mean_test)[1]
 
                p_in_out[i] = np.min([p_v[i],p_m[i],p_t])
                max_in_out[i] = np.max([v_larger[i],m_larger[i],t_larger])
                idx_p_min[i] = np.argmin([p_v[i],p_m[i],p_t])

                tr_min[i],tr_max[i] = np.min([mean_visual,mean_mem,mean_bl]),np.max([mean_visual,mean_mem,mean_bl])
                mean_mem_all[i],mean_visual_all[i] = mean_mem.mean(),mean_visual.mean()
        if ~np.all(np.isnan(tr_min)):
            tr_min_all = np.nanmin(tr_min)
            tr_max_all = np.nanmax(tr_max)-tr_min_all
            mean_mem = (mean_mem_all[0]-tr_min_all)/tr_max_all
            mean_visual = (mean_visual_all[0]-tr_min_all)/tr_max_all
            vm_index = (mean_mem-mean_visual)/(mean_mem+mean_visual)

        if (
            (p_in_out[0] is None or p_in_out[0] > 0.05)
            and (p_in_out[1] is None or p_in_out[1] < 0.05)
            and [v_larger[1], m_larger[1], t_larger][idx_p_min[1]]
        ):
            true_in_out='out'
            mean_mem = (mean_mem_all[1]-tr_min_all)/tr_max_all
            mean_visual = (mean_visual_all[1]-tr_min_all)/tr_max_all
            vm_index = (mean_mem-mean_visual)/(mean_mem+mean_visual)            

        for i,in_out in enumerate(["in","out"]): # iterate by code'

            neurons_info['array_position']+=[i_neuron]
            neurons_info['cluster']+=[n_type]
            neurons_info['group']+=[type_neuron]            
            neurons_info['in_out']+=[in_out]
            neurons_info['true_in_out']+=[true_in_out]
            neurons_info['vm_index']+=[vm_index]
            neurons_info['p']+=[p_in_out[1]]
            neurons_info['larger']+=[larger[i]]  
            neurons_info['v_larger']+=[v_larger[i]]  
            neurons_info['p_v']+=[p_v[i]]  
            neurons_info['m_larger']+=[m_larger[i]]  
            neurons_info['p_m']+=[p_m[i]]  
            
   
    neurons_info = pd.DataFrame(neurons_info)
    return neurons_info

### Start analysis

In [18]:
# load data
filepath = "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/lip/spikes/2022-12-05_10-51-52_Riesling_lip_e1_r1_sp.h5"  #2023-02-27_10-50-03 2023-02-09_10-42-54 2023-03-21_10-40-02
bhv_path = '/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/bhv/2022-12-05_10-51-52_Riesling_e1_r1_bhv.h5'
s_path = os.path.normpath(filepath).split(os.sep)
data = SpikeData.from_python_hdf5(filepath)
bhv = BhvData.from_python_hdf5(bhv_path)

In [19]:
# Select trials and create task frame
block = 1
trials_block = np.where(np.logical_and(data.trial_error == 0, data.block == block))[0]
print("Number of clusters: %d" % len(data.clustersgroup))

Number of clusters: 19


In [20]:
if np.any(np.isnan(data.neuron_cond)):
    neuron_cond = np.ones(len(data.clustersgroup))
else:
    neuron_cond = data.neuron_cond
task = def_task.create_task_frame(
    condition=bhv.condition[trials_block],
    test_stimuli=bhv.test_stimuli[trials_block],
    samples_cond=task_constants.SAMPLES_COND,
    neuron_cond=neuron_cond,
)

In [21]:
sp_samples = data.sp_samples
neuron_type = data.clustersgroup
code_samples = data.code_samples
code_numbers = data.code_numbers
sp_samples = data.sp_samples

fix_t = 200
dur_v = 200
st_m = 450 #800
end_m = 900
st_t = 900
end_t = 900+450
min_trials = 3
n_spikes_sec = 5
p_threshold = 0.05
vm_threshold = 0.4

In [22]:
code_numbers[:,4]  

array([ 36., 127., 122., ...,  23.,  97.,  97.])

In [23]:
shifts_on = code_samples[:,4]  
shifts_on = shifts_on[:,np.newaxis]
shifts_on = np.where(np.isnan(shifts_on),0,shifts_on)
shifts_test = code_samples[:,6]
shifts_test = shifts_test[:,np.newaxis]
shifts_test = np.where(np.isnan(shifts_test),0,shifts_test)

sp_shift_on = SpikeData.indep_roll(sp_samples, -(shifts_on-fix_t).astype(int), axis=2)[:,:,:end_m+fix_t]
sp_shift_test = SpikeData.indep_roll(sp_samples, -(shifts_test).astype(int), axis=2)[:,:,:450]
sp_shift = np.concatenate([sp_shift_on,sp_shift_test],axis=2)

#### Select responding neurons and check rf

In [24]:
task

,i_neuron,trial_idx,sample,in_out,n_test_stimuli,code
0,0,3,o1_c1,in,4,4
1,0,4,o1_c1,in,5,6
2,0,10,o1_c1,in,4,4
3,0,26,o1_c1,in,5,5
4,0,31,o0_c0,in,5,7
...,...,...,...,...,...,...
9134,18,465,o5_c5,out,3,3
9135,18,466,o5_c5,out,4,4
9136,18,467,o5_c5,out,1,1
9137,18,468,o5_c5,out,1,1


In [25]:
neurons_info = get_neurons_info(sp_shift,fix_t,neuron_type,task,dur_v=dur_v,st_m=st_m,end_m=end_m,st_t=st_t,end_t=end_t,min_trials=min_trials,n_spikes_sec=n_spikes_sec)
neurons_info = neurons_info[(neurons_info['p']<0.05)& (neurons_info['larger']==True)] # responding neurons


#### Compute vm index

In [26]:
neurons_info = neurons_info[neurons_info["in_out"]==neurons_info["true_in_out"]]
neurons_info = neurons_info[(np.logical_and(neurons_info["p_v"]<0.05,neurons_info["v_larger"]==True)) | (np.logical_and(neurons_info["p_m"]<0.05,neurons_info["m_larger"]==True))]

In [27]:
neurons_info

,array_position,cluster,group,in_out,true_in_out,vm_index,p,larger,v_larger,p_v,m_larger,p_m
1,0,1,mua,out,out,-0.215190,3.102793e-02,True,True,3.102793e-02,True,0.298736
8,4,2,good,in,in,-0.506667,5.959208e-08,True,True,1.465508e-09,True,0.451600
12,6,3,good,in,in,0.039151,2.074436e-20,True,True,2.869280e-01,True,0.000011
16,8,4,good,in,in,0.625000,2.732746e-04,True,False,9.088256e-01,True,0.000005
24,12,7,mua,in,in,-0.050100,1.459300e-11,True,True,2.196985e-08,True,0.000003
28,14,8,mua,in,in,0.294118,1.095396e-03,True,True,3.430967e-01,True,0.000006
36,18,9,good,in,in,0.196123,1.259793e-06,True,False,1.843076e-03,True,0.000177
